In [39]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.svm import NuSVR
import numpy as np

In [58]:
training_set = pd.read_csv("../data/preprocessed_input_interpolate_20min_phase1and2_train.csv")
predict_set = pd.read_csv("../data/preprocessed_input_interpolate_20min_phase2_test.csv")
predict_set = predict_set[lambda df : ((df.hour >= 6) & (df.hour < 8)) | ((df.hour >= 15) & (df.hour < 17))]
training_set = training_set.set_index(['date'])
predict_set = predict_set.set_index(['date'])

# Only select columns which are useful
selected_col = ['hour', 'wind_speed']

# Target predict columns
predict_col = "('C', 3)"
int_tol_id = "C,3," # For output csv
predict_col_shifted = "predict_shift_6"

# Time shift backward for predict_col
training_set_shift_6 = training_set.shift(periods=-6)[predict_col].fillna(method="ffill")

# Add time shifted predict_col to dataset
training_set = training_set.assign(predict_shift_6=pd.Series(training_set_shift_6).values)

In [59]:
clf = SVR(C=1.0, epsilon=0.1)
clf.fit(training_set[selected_col], training_set[predict_col_shifted])
final_predicted = clf.predict(predict_set[selected_col])

In [60]:
f = open('SVR_' + predict_col + '.csv', 'w')

csv_context = ""
# csv_context += "intersection_id,tollgate_id,time_window,avg_travel_time\n"
day = ["25", "26", "27", "28", "29", "30", "31"]
hour = ["08", "09", "17", "18"]
add_hour = ["10", "19"]
minute = ["00", "20", "40"]
d, h, m = 0, 0, 0
row_num = 0
for row in final_predicted:
    csv_context += int_tol_id + "\"[2016-10-" + day[d] + " " + hour[h] + ":" + minute[m] + ":00,2016-10-"
    if row_num != 0 and row_num % 6 == 2:
        h = (h + 1) % len(hour)
    m = (m + 1) % len(minute)
    hour_text = add_hour[int((h - 1) / 2)] if (row_num != 0 and row_num % 6 == 5) else hour[h]
    csv_context += day[d] + " " + hour_text + ":" + minute[m] + ":00)" "\"," + str(row) + "\n"
    if row_num != 0 and row_num % 6 == 5:
        h = (h + 1) % len(hour)
    if row_num != 0 and row_num % 12 == 11:
        d = (d + 1) % len(day)
    row_num += 1

f.write(csv_context)
f.close()